In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

In [ ]:
with open('data/config.txt', 'r') as json_file:
    data = json.load(json_file)

In [ ]:
df1_archive = pd.read_csv("data/fire_archive_V1_96617.csv")
df1_real = pd.read_csv("data/fire_nrt_V1_96617.csv")
df2_archive = pd.read_csv("data/fire_archive_V1_96617.csv")
df2_real = pd.read_csv("data/fire_nrt_V1_96617.csv")

In [ ]:
df1_archive = df1_archive.sort_values("acq_date")
df1 = df1_archive

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

MAPBOX_TOKEN = data["token"]
MAPBOX_STYLE = "mapbox://styles/mapbox/dark-v10"

In [ ]:
def to_hour_str(number):
    string = '{:04d}'.format(number)
    string = string[:2] + ':' + string[2:]
    return string

In [ ]:
df1['acq_datestring'] = df1.apply(lambda r: r['acq_date'] + " " + to_hour_str(r['acq_time']), axis=1)

In [ ]:
df1["lat_int"] = pd.cut(df1["latitude"], 1000).astype(str)
df1["long_int"] = pd.cut(df1["longitude"], 1000).astype(str)

In [ ]:
df1["long_int"].dtype

In [ ]:
fig = px.histogram(df1, x="latitude")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df1["longitude"],
    y=df1["latitude"],
    mode='markers',
    showlegend=False,
    marker=dict(
        symbol='circle',
        opacity=0.7,
        color='white',
        size=8,
        line=dict(width=1),
    )
))

fig.add_trace(go.Histogram2d(
    x=df1["longitude"],
    y=df1["latitude"],
    colorscale="Viridis",
    nbinsx=1400,
    nbinsy=1400,
    zauto=False,
))
fig.show()


In [ ]:
times = df1_archive.groupby(['acq_date'])['acq_date'].count().index.tolist()
frames_data = [df1_archive.loc[df1_archive['acq_date'] == t] for t in times]

In [ ]:
df = df1_archive[df1_archive["acq_date"] == "2019-08-01"]

In [ ]:
print(df[["latitude","longitude"]])

In [ ]:
frames= [go.Frame(data=[go.Densitymapbox(lat=f['latitude'], lon=f['longitude'], z=f['bright_ti4'], radius=10)], name=str(f.iloc[0]['acq_date'])) for f in frames_data]

In [ ]:
buttons=[
         dict(label="Play",method="animate",args=[None, {'fromcurrent':True, "transition": {"duration": 30, "easing": "quadratic-in-out"}}]),
         dict(label="Pause",method="animate",args=[[None], {"frame": {"duration": 0, "redraw": False},"mode": "immediate", "transition": {"duration": 0}}])
]


sliders_dict = {
    'active':0,
    'currentvalue': dict(font=dict(size=16), prefix='Time: ', visible=True),
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    'x': 0,
    'steps': []
}

for i,t in enumerate(times):
    slider_step = {"args": [
                        [t],
                        {"frame": {"duration": 300, "redraw": False},
                         #"mode": "immediate",
                         "transition": {"duration": 30, "easing": "quadratic-in-out"}}
                    ],
            "label": t,
            "method": "animate",
            "value": t
    }
    sliders_dict['steps'].append(slider_step)

In [ ]:
fig = go.Figure(data = [go.Densitymapbox(lat=df1_archive['latitude'], lon=df1_archive['longitude'], z=df1_archive['bright_ti4'], radius=1, colorscale='Hot', zmax=400, zmin=0)],
               layout=go.Layout(updatemenus=[dict(type="buttons", buttons=buttons,showactive=True)] ), 
               frames=frames
)

fig.update_layout(mapbox_style=MAPBOX_STYLE, 
                  mapbox_accesstoken=MAPBOX_TOKEN,
                  mapbox_center_lon=135,
                  mapbox_center_lat=-25.34,
                  mapbox_zoom=3.5)

"""fig.update_layout(mapbox_style="stamen-terrain", 
                  mapbox_center_lon=135,
                  mapbox_center_lat=-25.34,
                  mapbox_zoom=3.5)"""


fig.update_layout(sliders=[sliders_dict],
                 title='Australia fires over time')

fig.update_layout(height=850)
fig.show()